In [1]:
import numpy as np
import json
import string

In [2]:
file = open('recipes.json') 
recipes = json.load(file)

In [3]:
def tokenisation(input_string):
    
    new_string = input_string.translate(str.maketrans(string.digits, ' '*len(string.digits)))
    new_string = new_string.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    new_string = new_string.lower()
    
    return new_string

In [4]:
def squash(entry):
    
    unique_entries = set(entry)
    return sorted([(word, entry.count(word)) for word in unique_entries])

In [5]:
def search(recipe_dict):
    
    sorted_dict = {}
    counter = 0
    key_list = list(recipe_dict.keys())
    value_list = list(recipe_dict.values())
    
    for value in value_list:
        
        splitted_string = tokenisation(str(value)).split()
        
        for word in splitted_string:
            
            if len(word) < 3:
                continue
                
            elif word not in sorted_dict:
                sorted_dict[word] = []
                sorted_dict[word].append(key_list[counter])
                
            else:
                sorted_dict[word].append(key_list[counter])
                
        counter += 1
    
    return sorted_dict

In [6]:
def location(dictionary):

    storage = []
    searched_dict = search(dictionary)
    
    list_d = list(searched_dict)
    sorted_list = sorted(list_d)
    
    for i in sorted_list:
        
        storage.append((i, squash(searched_dict[i])))
        
    return storage

In [7]:
def normal_score(dictionary):
    
    recipes = location(dictionary)    
    word_list = list(np.array(recipes)[:,0])
    loc_list = list(np.array(recipes)[:,1])
    score_list = []
    
    for i in loc_list:
        
        score = 0
        
        for j in range(len(i)):
            
            if i[j][0] == 'title':
                score += 8 * i[j][1]
                
            elif i[j][0] == 'categories':
                score += 4 * i[j][1]      
                
            elif i[j][0] == 'ingredients':
                score += 2 * i[j][1]
                
            elif i[j][0] == 'directions':
                score += 1 * i[j][1]
                
        score_list.append(score)
        
    if 'rating' in dictionary:
        score_list.append(dictionary['rating'])
        
    return list((word_list, score_list))

In [8]:
def index_normal(dictionary):
    
    index_dictionary = {}
    score_value = normal_score(dictionary)
    recipe_title = dictionary['title']
    index_dictionary.update({recipe_title:score_value})
    
    return index_dictionary 

In [9]:
def simple_score(dictionary):
    
    recipes = location(dictionary)
    word_list = list(np.array(recipes)[:,0])
    recipe_score = 0

    ing_count = len(dictionary['ingredients'])
    dir_count = len(dictionary['directions'])

    if ing_count <= 1 or dir_count <= 1:
        recipe_score = 10000000    
    else:
        recipe_score +=ing_count*dir_count
    
    return word_list, recipe_score

In [10]:
def index_simple(dictionary):

    index_dictionary = {}
    score_value = simple_score(dictionary)
    recipe_title = dictionary['title']
    index_dictionary.update({recipe_title:score_value})
    
    return index_dictionary 

In [11]:
def health_score(dictionary):
    
    recipes = location(dictionary)
    word_list = list(np.array(recipes)[:,0])
    recipe_score = 1000000
    
        
    cal = dictionary['calories']
    pro = dictionary['protein']
    fat = dictionary['fat']        
        
    for n in range(1,5):
        new_score = (abs(cal - 510*n))/510 + 2*(abs((pro - 18*n))/18) + 4*(abs((fat - 150*n))/150)

        if new_score < recipe_score:
            recipe_score = new_score
               
    return word_list, recipe_score

In [12]:
def index_health(dictionary):
    
    index_dictionary = {}
    score_value = health_score(dictionary)
    recipe_title = dictionary['title']
    index_dictionary.update({recipe_title:score_value})
    
    return index_dictionary   

In [13]:
normal_dict = {}

for i in recipes:
    normal_dict.update(index_normal(i))
    
simple_dict = {}

for i in recipes:
    
    if 'ingredients' in i and 'directions' in i:
        simple_dict.update(index_simple(i))
    
healthy_dict = {}

for i in recipes:
    
    if 'calories' in i and 'fat' in i and 'protein' in i:
        healthy_dict.update(index_health(i))

In [14]:
def search(query, ordering = 'normal', count = 10):
    
    token_list = tokenisation(query)
    tokens = list(token_list.split())

    for i in tokens:

        if len(i) < 3:
            tokens.remove(i)
            
    score_dict = {}
    
    if ordering == 'normal':

        for j, k in normal_dict.items():
            
            score = 0

            if all(x in k[0] for x in tokens):

                for l in range(len(tokens)):

                    word_loc = k[0].index(tokens[l])
                    score += k[1][word_loc]

                if len(k[0]) != len(k[1]):
                    score += k[1][-1]

                score_dict.update({j:score})

        if len(score_dict) < count:
            normal = sorted(score_dict.items(), key = lambda x: x[1], reverse = True)[:len(score_dict)]

            for recipe in normal:
                print(recipe[0])
            
        else:
            normal = sorted(score_dict.items(), key = lambda x: x[1], reverse = True)[:count]

            for recipe in normal:
                print(recipe[0])
                
    if ordering == 'simple':

        new_recipe_dict = [score_dict.update({j:k[1]}) for j, k in simple_dict.items() if all(token in k[0] for token in tokens)]          

        if len(score_dict) < count:          
            simple = sorted(score_dict.items(), key = lambda x: x[1])[:len(score_dict)]

            for recipe in simple:
                print(recipe[0])

                
        else:
            simple = sorted(score_dict.items(), key = lambda x: x[1])[:count]

            for recipe in simple:
                print(recipe[0])

    if ordering == 'healthy':

        new_recipe_dict = [score_dict.update({j:k[1]}) for j, k in healthy_dict.items() if all(token in k[0] for token in tokens)]

        if len(score_dict) < count:
            healthy = sorted(score_dict.items(), key = lambda x: x[1])[:len(score_dict)]

            for recipe in healthy:
                print(recipe[0])  
            
        else:
            healthy = sorted(score_dict.items(), key = lambda x: x[1])[:count]

            for recipe in healthy:
                print(recipe[0])

In [18]:
search('tempura', 'healthy', 10)

Sesame Tempura Green Beans with Soy Dipping Sauce 
Shrimp Tempura with Lemon and Olive Mayonnaise 
Sweet Potato Tempura and Steak Sandwich with Bok Choy and Miso Mayo 
Shrimp Tempura Cocktail 
Heaven-and-Earth Tempura Cakes 
Sugar Snap Pea Tempura 
